In [1]:
import brightpath

In [10]:
fp="/Users/romain/OneDrive/Documents/Power2X/bw_ei39_120823.xlsx"

In [11]:
c = brightpath.Converter(
    filepath=fp,
    metadata="/Users/romain/OneDrive/Documents/Power2X/meta_power2X_simapro.yaml",
    export_dir="/Users/romain/OneDrive/Documents/Power2X"
)

Extracted 8 worksheets in 1.04 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.24 seconds


In [12]:
c.convert_to_simapro(database="uvek")

The following exchanges have not been used:
+----------------------------------------+----------+----------+------------+
|                Exchange                |  Amount  | Location | Categories |
+----------------------------------------+----------+----------+------------+
|         market for polysulfone         | kilogram |   GLO    |    None    |
|     market for strontium carbonate     | kilogram |   GLO    |    None    |
| market for fly ash and scrubber sludge | kilogram |    CH    |    None    |
|        market for barium oxide         | kilogram |   GLO    |    None    |
| market for activated carbon, granular  | kilogram |   GLO    |    None    |
|     market for molybdenum trioxide     | kilogram |   GLO    |    None    |
|                 Oxygen                 | kilogram |   GLO    |  ('air',)  |
+----------------------------------------+----------+----------+------------+


'Inventories export to: /Users/romain/OneDrive/Documents/Power2X/simapro_uvek_16-08-2023.csv'

In [13]:
import pandas as pd

fp="/Users/romain/OneDrive/Documents/Power2X/simapro_uvek_16-08-2023.csv"
exp_dir = "/Users/romain/OneDrive/Documents/Power2X/formatted_simapro_lci.xlsx"

# Function to parse individual lines
def parse_line_final_corrected(line, category):
    """Parses a line of data into various columns, corrected for resource flows, float precision, and amount extraction."""
    parts = line.split(";")
    name = parts[0] if len(parts) > 0 else None
    location = name.rsplit("/", 1)[1].split()[0] if "/" in name else "RER"
    category_name = category if name else None
    subcategory = "high population density" if "Emissions" in category else "-"
    infrastructure_process = "1" if "Products" in category else "0"
    unit = parts[1] if len(parts) > 1 else None
    product_name = None
    
    # Adjusting the position for resource flow amounts
    if "Resources" in category:
        amount = "{:.2e}".format(float(parts[3]))
    elif "Emissions" in category and len(parts) > 3:
        amount = "{:.2e}".format(float(parts[3]))
    elif len(parts) > 2:
        amount = "{:.2e}".format(float(parts[2]))
    else:
        amount = None
        
    uncertainty_type = 1 if name else None
    standard_deviation = None
    general_comment = parts[-1] if len(parts) > 3 else None
    return name, location, category_name, subcategory, infrastructure_process, unit, product_name, amount, uncertainty_type, standard_deviation, general_comment

# Function to parse the entire dataset into a DataFrame
def parse_dataset_to_df_updated(dataset):
    """Parses a dataset into a DataFrame with the specified format, adding headers."""
    categories = {
        "Products": "Output",
        "Materials/fuels": "Material inputs",
        "Electricity/heat": "Energy inputs",
        "Resources": "Resources",
        "Emissions to air": "Emissions to air",
        "Emissions to water": "Emissions to water",
        "Emissions to soil": "Emissions to soil",
        "Waste to treatment": "Waste treatment",
        "Final waste flows": "Final waste flows",
        "Non material emission": "Non material emission",
        "Social issues": "Social issues",
        "Economic issues": "Economic issues",
        "End": None
    }
    
    parsed_data = []
    current_category = None
    
    for line in dataset:
        line = line.strip()
        if line in categories:
            current_category = line
            if categories[current_category] is not None:
                parsed_data.append((categories[current_category], None, None, None, None, None, None, None, None, None, None))
                categories[current_category] = None
        elif current_category and line:
            parsed_line = parse_line_final_corrected(line, current_category)
            parsed_data.append(parsed_line)
        elif not line and parsed_data:
            parsed_data.append((None, None, None, None, None, None, None, None, None, None, None))
    
    df = pd.DataFrame(parsed_data, columns=["Name", "Location", "Category", "SubCategory", "InfrastructureProcess", "Unit", "Product", "Amount", "UncertaintyType", "StandardDeviation95%", "GeneralComment"])
    return df

# Splitting the uploaded file into individual datasets
with open(fp, "r") as file:
    lines = file.readlines()

datasets = []
current_dataset = []
for line in lines:
    if "Process:" in line:
        if current_dataset:
            datasets.append(current_dataset)
        current_dataset = [line]
    else:
        current_dataset.append(line)
datasets.append(current_dataset)

# Parsing and saving the datasets into an Excel file
all_tables_final_corrected = pd.DataFrame()

for i, dataset in enumerate(datasets):
    df = parse_dataset_to_df_updated(dataset)
    df.index = df.index + len(all_tables_final_corrected) + 2
    all_tables_final_corrected = pd.concat([all_tables_final_corrected, df])
    all_tables_final_corrected = pd.concat([all_tables_final_corrected, pd.DataFrame([["","","","","","","","","","",""]], columns=["Name", "Location", "Category", "SubCategory", "InfrastructureProcess", "Unit", "Product", "Amount", "UncertaintyType", "StandardDeviation95%", "GeneralComment"])])

all_tables_final_corrected.to_excel(exp_dir, index=False)
